# Sparse Correspondence

The notebooks in this folder replicate the experiments as performed for [CNNs on Surfaces using Rotation-Equivariant Features](https://doi.org/10.1145/3386569.3392437).

The current notebook replicates the shape segmentation experiments from section `5.2 Comparisons`.

## Imports
We start by importing dependencies.

In [1]:
# File reading and progressbar
import os.path as osp
import progressbar

import numpy as np
import random
import time

# PyTorch and PyTorch Geometric dependencies
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.data import DataLoader
from torch_geometric.nn.inits import zeros
from torch import autograd
import os
import trimesh as tm
import vectorheat as vh
#os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

#tm.SHRECGTtoVertex()
#tm.transferSHRECGT()
#tm.splitsSHREC19()

# Harmonic Surface Networks components
# Layers
from nn import (ECResNetBlock,LiftBlock, ParallelTransportPool, ParallelTransportUnpool,
                TwinLoss, TwinEval, TangentPerceptron,
                TangentLin, TangentNonLin, VectorDropout, ECHOBlock, ExtConvCplx)
# Utility functions
from utils.harmonic import magnitudes, norm2D
# Rotated MNIST dataset
from datasets import SHREC19PR
# Transforms
from transforms import (HarmonicPrecomp, VectorHeat, MultiscaleRadiusGraph, 
                        ScaleMask, FilterNeighbours, NormalizeArea, NormalizeAxes, Subsample)

## Settings
Next, we set a few parameters for our network. You can change these settings to experiment with different configurations of the network. Right now, the settings are set to the ones used in the paper.

In [2]:
# Band-limit for extended convolution
band_limit = 1

# Number of rings in the radial profile
n_corr_rings = 6

# Number of conv rings
n_conv_rings = 6

# Learn radial offset for correlations
offset = True;

# Number of filters per block
nf = [16, 32];

n_des = 16;
n_bins = 2;

# Ratios used for pooling
ratios=[1, 0.25]

# Radius of convolution for each scale
radii = [0.1, 0.2]

# Output descriptor dimension
desDim = 16;


# Number of datasets per batch
batch_size = 1

bias = False;
smooth = False;


## Dataset
To get our dataset ready for training, we need to perform the following steps:
1. Provide a path to load and store the dataset.
2. Define transformations to be performed on the dataset:
    - A transformation that computes a multi-scale radius graph and precomputes the logarithmic map.
    - A transformation that masks the edges and vertices per scale and precomputes convolution components.
3. Assign and load the datasets.

In [3]:
# 1. Provide a path to load and store the dataset.
# Make sure that you have created a folder 'data' somewhere
# and that you have downloaded and moved the raw datasets there
path = osp.join('data', 'SHREC19PR')

# 2. Define transformations to be performed on the dataset:
# Transformation that computes a multi-scale radius graph and precomputes the logarithmic map.
pre_transform = T.Compose((
    #NormalizeArea(),
    MultiscaleRadiusGraph(ratios, radii, loop=True, flow='target_to_source', sample_n=2048),
    VectorHeat(max_lvl=len(ratios)-1),
    Subsample(),
))
# Apply a random scale and random rotation to each shape
#transform = None

transform = T.Compose((
    T.Center(),
    T.RandomRotate(45, axis=0),
    T.RandomRotate(45, axis=1),
    T.RandomRotate(45, axis=2)
)
)

# Transformations that masks the edges and vertices per scale and precomputes convolution components.
scale0_transform = T.Compose((
    ScaleMask(0),
    FilterNeighbours(radii[0]),
    HarmonicPrecomp(n_conv_rings, n_corr_rings, band_limit, max_r=radii[0]))
)

scale1_transform = T.Compose((
    ScaleMask(1),
    FilterNeighbours(radii[1]),
    HarmonicPrecomp(n_conv_rings, n_corr_rings, band_limit, max_r=radii[1]))
)


# 3. Assign and load the datasets.
trainS_dataset = SHREC19PR(path, 0, pre_transform=pre_transform, transform=transform)
trainT_dataset = SHREC19PR(path, 1, pre_transform=pre_transform, transform=transform)

testS_dataset = SHREC19PR(path, 2, pre_transform=pre_transform)
testT_dataset = SHREC19PR(path, 3, pre_transform=pre_transform)



## Network architecture
Now, we create the network architecture by creating a new `nn.Module`, `Net`. We first setup each layer in the `__init__` method of the `Net` class and define the steps to perform for each batch in the `forward` method. The following figure shows a schematic of the architecture we will be implementing:

<img src="img/resnet_architecture.png" width="800px" />

Let's get started!

In [4]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.lin0 = nn.Linear(3, nf[0])

        self.lift = LiftBlock(3, nf[0], n_corr_rings, offset, MLP=False)
                
        self.resnet1 = ECResNetBlock(nf[0], nf[1], band_limit, n_conv_rings)
                
        self.resnet2 = ECResNetBlock(nf[1], nf[1], band_limit, n_conv_rings)
        
        self.resnet3 = ECResNetBlock(nf[1], nf[1], band_limit, n_conv_rings)
        
        self.resnet4 = ECResNetBlock(nf[1], nf[1], band_limit, n_conv_rings)

        self.resnet5 = ECResNetBlock(nf[1], nf[1], band_limit, n_conv_rings)
        
        self.resnet6 = ECResNetBlock(nf[1], nf[1], band_limit, n_conv_rings)
        
        self.resnet7 = ECResNetBlock(nf[1], nf[1], band_limit, n_conv_rings)
        
        self.resnet8 = ECResNetBlock(nf[1], nf[0], band_limit, n_conv_rings, back=True)
        
        self.conv_final = ExtConvCplx(nf[0], nf[0], 1, n_conv_rings)

        
        #self.echo = ECHOBlock(nf[0], n_des, n_bins, desDim, band_limit, n_conv_rings, classify=False, mlpC=[128, 64])

        #self.D = nn.Dropout(p=0.5)
        
        self.d = VectorDropout(p=0.0)

        
        self.res1 = TangentPerceptron(nf[0], nf[1])
        
        
        self.res2 = TangentPerceptron(nf[1], nf[1])
        
        self.res3 = TangentPerceptron(nf[1], nf[1])
                
        self.res4 = TangentPerceptron(nf[1], nf[0])
        
        self.lin = TangentPerceptron(nf[1], nf[0])



        
        # Pool
        self.pool = ParallelTransportPool(1, scale1_transform)
        self.unpool = ParallelTransportUnpool(from_lvl=1)


    def forward(self, data):
        
        ###############
        ### Level 1 ###
        ###############
        
        data_scale0 = scale0_transform(data)

        
        
        attr_grad = (data_scale0.edge_index, data_scale0.pcmp_gather)

        attr_conv = (data_scale0.edge_index, data_scale0.pcmp_scatter, 
                      data_scale0.connection)    
        
        attr_echo = (data_scale0.edge_index, data_scale0.pcmp_scatter, 
                     data_scale0.pcmp_echo, data_scale0.connection)
 
        x = data.pos

        x1 = self.lift(x, *attr_grad)
                                        
        x = self.resnet1(x1, *attr_conv) 
        
        x2 = self.resnet2(x, *attr_conv)  + self.res1(x1)
                
        x = self.resnet3(x2, *attr_conv)
        
        x3 = self.resnet4(x, *attr_conv) + self.res2(x2)
        
        x = self.resnet5(x, *attr_conv)
        
        x4 = self.resnet6(x, *attr_conv) + self.res3(x3)
        
        x = self.resnet7(x4, *attr_conv)
        
        x = self.resnet8(x, *attr_conv) + self.res4(x4)
        
       # x = self.echo(x, *attr_echo)
        return norm2D(x)

    

## Training

Phew, we're through the hard part. Now, let's get to training. First, move the network to the GPU and setup an optimizer.

In [5]:
# We want to train on a GPU. It'll take a long time on a CPU
device = torch.device('cuda')
# Move the network to the GPU
model = Net().to(device)
statePath = '/home/tommy/Dropbox/specialMath/Harmonic/ECHONet/V7/plot/Figures/SHREC19PR/states/VFC_16_3'
model.load_state_dict(torch.load(statePath))
model = model.to(device)
# Set up the ADAM optimizer with learning rate of 0.0076 (as used in H-Nets)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

decay = 0.975

train_size = 80;
test_size = 20;

n_L_pairs = 512
mu = 5
ratio = 0.5
twinL = TwinLoss(mu=mu)
twinE = TwinEval(mu=mu)

In [6]:


def getNullPairs(pos_pairs, nSamples=2048):
    pos_lin = pos_pairs[0] * nSamples + pos_pairs[1]
    all_lin = torch.arange(nSamples * nSamples)
    null_lin = torch.from_numpy(np.setdiff1d(all_lin.cpu().numpy(), pos_lin.cpu().numpy())).long()

    npS = torch.remainder(null_lin, nSamples)
    npT = torch.div(torch.sub(null_lin, npS), nSamples)
    
    return torch.cat( (npT[..., None], npS[..., None]), dim=1).long()



Next, define a training and test function.

In [7]:
def train(epoch, acc=0):
    # Set model to 'train' mode
    model.train()

  
    if epoch >= 25:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.001;
   
            
    # Sort out progress bar
    n_data = trainS_dataset.__len__()
    widgets = [progressbar.Percentage(), progressbar.Bar(), 
              progressbar.AdaptiveETA(), ' | Loss:', progressbar.Variable('loss'),]

    bar = progressbar.ProgressBar(max_value=n_data, widgets=widgets)

    order = torch.randperm(n_data).long()
    
    totalL = 0.0;
    
    for i in range(n_data):
    # Move training data to the GPU and optimize parameters
        #with autograd.detect_anomaly():
        
        optimizer.zero_grad()
        
        dataS = trainS_dataset.get(order[i])
        dataT = trainT_dataset.get(order[i])

        dataT.null_pairs = getNullPairs(dataT.pos_pairs, dataT.pos.size(0))
        
        FS = model(dataS.to(device))
        FT = model(dataT.to(device))
        

        p_ = torch.randperm(dataT.pos_pairs.size(0))[:n_L_pairs];
        n_ = torch.randperm(dataT.null_pairs.size(0))[:n_L_pairs]
        
        L = twinL(FS, FT, dataT.pos_pairs[p_, :], dataT.null_pairs[n_, :])
                
        totalL += L.item()
        i = i + 1
        bar.update(i, loss = (totalL / i) )
        L.backward()

        optimizer.step()

        
        


Train for 50 epochs.

In [8]:
def test():
    # Set model to 'evaluation' mode
    model.eval()
    
    n_false_null = 0;
    n_false_pos = 0;
    
    n_p = 0;
    n_n = 0;
        
    n_test = testS_dataset.__len__();
    for i in progressbar.progressbar(range(n_test)):
        with torch.no_grad():
            dataS = testS_dataset.get(i)
            dataT = testT_dataset.get(i)
            
            dataT.null_pairs = getNullPairs(dataT.pos_pairs, dataT.pos.size(0))
        



            FS = model(dataS.to(device))
            FT = model(dataT.to(device))

            nFP, nFN = twinE(FS, FT, dataT.pos_pairs, dataT.null_pairs)

            n_false_pos += nFP
            n_false_null += nFN

            n_p += dataT.pos_pairs.size(0)
            n_n += dataT.null_pairs.size(0)
   

    rateFP = n_false_pos / n_p
    rateFN = n_false_null / n_n
        
    return rateFP, rateFN

In [9]:
#statePath = '/home/tommy/Dropbox/specialMath/Harmonic/ECHONet/V7/plot/Figures/SHREC19PR/states/VFC_16_3'
#model.load_state_dict(torch.load(statePath))


In [10]:
print('Start training, may take a while...')
# Try with fewer epochs if you're in a timecrunch
acc = 0;
best=1e12;
for epoch in range(45, 50):
    train(epoch, acc)
    if (epoch % 5) == 0 and epoch > 0:
        rateFP, rateFN = test() 
        classE = rateFP + rateFN;
        torch.save(model.state_dict(), statePath.format(epoch))
        # if classE < best:
            #best = classE
            #torch.save(model.state_dict(), statePath.format(best))
        print("Epoch {} - FP: {:06.4f}, FN: {:06.4f}, Err: {:06.4f} ".format(epoch, rateFP, rateFN, rateFP + rateFN))

Start training, may take a while...


  3%|#                                      |ETA:   0:02:12 | Loss:loss:   4.58

KeyboardInterrupt: 

In [ ]:
    
## Plot
#statePath = '/home/tommy/Dropbox/specialMath/Harmonic/ECHONet/V7/plot/Figures/SHREC19PR/temp/VFC_16_5'

#torch.save(model.state_dict(), statePath)        


In [ ]:
from torch_geometric.io import read_ply, read_off, read_obj
import scipy as sp

def curvePR(saveFile=None):
    # Set model to 'evaluation' mode
    model.eval()
    
    n_test = testS_dataset.__len__();
    
    nPoints = testS_dataset.get(0).pos.size(0)
    
    meanPR = torch.Tensor(2, nPoints).fill_(0)
    
    nCurves = 0;
    nNAN = 0;
    for i in progressbar.progressbar(range(n_test)):
        with torch.no_grad():
            dataS = testS_dataset.get(i)
            dataT = testT_dataset.get(i)

            FS = model(dataS.to(device))
            FT = model(dataT.to(device))
            
            for l in range(nPoints):
                
                indST = torch.nonzero(dataT.pos_pairs[:, 1] == l).squeeze(1)
                
                if (indST.size(0) > 0):

                    mST = dataT.pos_pairs[indST, 0]

                    dST = torch.sum(torch.pow(torch.sub(FT, FS[l, None, :]), 2), dim=1)

                    pr = torch.from_numpy(tm.computePR(dST.cpu().numpy(), mST.cpu().numpy()))
                
                    if torch.isnan(pr).any() == False:
                        meanPR[...] = meanPR[...] + pr[...]
                        nCurves = nCurves + 1
                    else:
                        nNAN = nNAN + 1;

    meanPR = torch.div(meanPR, nCurves)
    meanPR = meanPR.cpu().numpy()
    
    print('Num NaN = {}'.format(nNAN), flush=True)
    
    if saveFile is not None:
        np.save(saveFile, meanPR)
    
    return meanPR

In [ ]:
from torch_geometric.io import read_ply, read_off, read_obj
import scipy as sp

def curvePR2(saveFile=None, alpha = 0.05):
    # Set model to 'evaluation' mode
    model.eval()
    
    n_test = testS_dataset.__len__();
    
    nPoints = testS_dataset.get(0).pos.size(0)
    
    meanPR = torch.Tensor(2, nPoints).fill_(0)
    
    modelPath = '/home/tommy/Dropbox/specialMath/Harmonic/ECHONet/V7/data/SHREC19PR/raw/models_eval/{}.obj'

    
    
    nCurves = 0;
    nNAN = 0;
    for i in progressbar.progressbar(range(n_test)):
        with torch.no_grad():
            dataS = testS_dataset.get(i)
            dataT = testT_dataset.get(i)

            gtInd = dataS.sample_idx[dataT.pos_pairs[:,1]]

            FS = model(dataS.to(device))
            FT = model(dataT.to(device))

            ## Load original mesh + gt
            dataS0 = read_obj(modelPath.format(dataS.name))
            posS = dataS0.pos.cpu().numpy()     
            facesS = dataS0.face.cpu().numpy().T
            areaS = vh.surface_area(posS, facesS)   
            posS = posS / np.sqrt(areaS)
            
            distMat = torch.from_numpy(tm.getAdjacentDist(posS, facesS, dataS.sample_idx.cpu().numpy()))
            
            for l in range(nPoints):
                
                local = torch.nonzero(distMat[l, :] <= alpha).squeeze(1)
                
                indST = torch.LongTensor();
                
                for k in range(local.size(0)):
                    indST = torch.cat( (indST, torch.nonzero(dataT.pos_pairs[:, 1] == local[k]).squeeze(1).cpu()), dim=0);
                
                #indST = torch.nonzero(dataT.pos_pairs[:, 1] == l).squeeze(1)
                
                if (indST.size(0) > 0):

                    mST = dataT.pos_pairs[indST, 0]

                    dST = torch.sum(torch.pow(torch.sub(FT, FS[l, None, :]), 2), dim=1)

                    pr = torch.from_numpy(tm.computePR(dST.cpu().numpy(), mST.cpu().numpy()))
                
                    if torch.isnan(pr).any() == False:
                        meanPR[...] = meanPR[...] + pr[...]
                        nCurves = nCurves + 1
                    else:
                        nNAN = nNAN + 1;

    meanPR = torch.div(meanPR, nCurves)
    meanPR = meanPR.cpu().numpy()
    
    print('Num NaN = {}'.format(nNAN), flush=True)
    
    if saveFile is not None:
        np.save(saveFile, meanPR)
    
    return meanPR

def allPR(savePath, alpha = 0.05):
    # Set model to 'evaluation' mode
    model.eval()
    
    n_test = testS_dataset.__len__();
    
    nPoints = testS_dataset.get(0).pos.size(0)
    
    meanPR = torch.Tensor(2, nPoints).fill_(0)
    
    modelPath = '/home/tommy/Dropbox/specialMath/Harmonic/ECHONet/V7/data/SHREC19PR/raw/models_eval/{}.obj'

    
    
    nCurves = 0;
    nNAN = 0;
    for i in progressbar.progressbar(range(n_test)):
        with torch.no_grad():
            dataS = testS_dataset.get(i)
            dataT = testT_dataset.get(i)

            gtInd = dataS.sample_idx[dataT.pos_pairs[:,1]]

            FS = model(dataS.to(device))
            FT = model(dataT.to(device))

            ## Load original mesh + gt
            dataS0 = read_obj(modelPath.format(dataS.name))
            posS = dataS0.pos.cpu().numpy()     
            facesS = dataS0.face.cpu().numpy().T
            areaS = vh.surface_area(posS, facesS)   
            posS = posS / np.sqrt(areaS)
            
            distMat = torch.from_numpy(tm.getAdjacentDist(posS, facesS, dataS.sample_idx.cpu().numpy()))
            
            prMat = torch.FloatTensor(nPoints, 2, nPoints).fill_(0)
            
            for l in range(nPoints):
                
                local = torch.nonzero(distMat[l, :] <= alpha).squeeze(1)
                
                indST = torch.LongTensor();
                
                for k in range(local.size(0)):
                    indST = torch.cat( (indST, torch.nonzero(dataT.pos_pairs[:, 1] == local[k]).squeeze(1).cpu()), dim=0);
                
                #indST = torch.nonzero(dataT.pos_pairs[:, 1] == l).squeeze(1)
                
                if (indST.size(0) > 0):

                    mST = dataT.pos_pairs[indST, 0]

                    dST = torch.sum(torch.pow(torch.sub(FT, FS[l, None, :]), 2), dim=1)

                    pr = torch.from_numpy(tm.computePR(dST.cpu().numpy(), mST.cpu().numpy()))
                
                    prMat[l, :, :] = torch.from_numpy(pr).float();
                    
                    meanPR[...] = meanPR[...] + pr[...]
                    nCurves = nCurves + 1
                else:
                    prMat[l, 0, 1] = -1;

            saveFile = savePath + '{0}.{1}.pr'.format(dataS.name, dataT.name)
            np.save(saveFile, prMat.cpu().numpy())
            
    meanPR = torch.div(meanPR, nCurves)
    
    print("zeroP = ", meanPR[0, 0], flush=True)


In [ ]:
from torch_geometric.io import read_ply, read_off, read_obj

def normalizedError(saveFile=None):
    # Set model to 'evaluation' mode
    model.eval()
    
    modelPath = '/home/tommy/Dropbox/specialMath/Harmonic/ECHONet/V7/data/SHREC19PR/raw/models_eval/{}.obj'
    gtPath = '/home/tommy/Dropbox/specialMath/Harmonic/ECHONet/V7/data/SHREC19PR/raw/gt/{0}.{1}.gt.txt'
    
    n_test = testS_dataset.__len__();
    for i in progressbar.progressbar(range(n_test)):
        with torch.no_grad():
            dataS = testS_dataset.get(i)
            dataT = testT_dataset.get(i)
            
            gtInd = dataS.sample_idx[dataT.pos_pairs[:,1]]

            FS = model(dataS.to(device))
            FT = model(dataT.to(device))

            ## Load original mesh + gt
            dataS0 = read_obj(modelPath.format(dataS.name))
            posS = dataS0.pos.cpu().numpy()     
            facesS = dataS0.face.cpu().numpy().T
            areaS = vh.surface_area(posS, facesS)   
            posS = posS / np.sqrt(areaS)

            ### Get indices of nearest descriptors
            nearestInd = torch.from_numpy(tm.getNearestDes(FS.cpu().numpy(), FT.cpu().numpy(), 
                                                            dataS.sample_idx[:,  None].cpu().numpy()).astype(int)).long()


            compInd = torch.cat( (nearestInd, gtInd[..., None]), dim=1)

            # Compute geodesic error
            error = tm.getGeoError(posS, facesS, compInd.cpu().numpy()).squeeze()

            if i == 0:
                E = torch.from_numpy(error).float()
            else:
                E = torch.cat((E, torch.from_numpy(error).float()), dim=0)   

    E, _ = torch.sort(E)

    E = E.cpu().numpy()
    
    if saveFile is not None:
        np.save(saveFile, E)
    
    return E

In [ ]:
'''
savePath = '/home/tommy/Dropbox/specialMath/Harmonic/ECHONet/V7/plot/SHREC19PR/'

ID = 'VFC_16_pr_3_2'

rawFile = savePath + ID + '_raw.npy'

plotFile = savePath + ID + '_plot.txt'
'''

savePath = '/home/tommy/Dropbox/specialMath/Harmonic/ECHONet/V7/plot/Figures/SHREC19PR/hitRate/FC/3/'
allPR(savePath)

In [ ]:
#statePath = '/home/tommy/Dropbox/specialMath/Harmonic/ECHONet/V7/plot/SHREC19PR/states/VFC_16_3_Final'
#model.load_state_dict(torch.load(statePath))

#prCurve = curvePR2(rawFile);
#E = normalizedError(rawFile);

In [ ]:
'''
prCurve = np.load(rawFile)

nSamples = 200;

x0 = prCurve[1, 0]

#print(prCurve[0, :5])

xVal = np.arange(0, nSamples+1) / nSamples;

xVal = (1 - x0)*xVal + x0;


fPR = sp.interpolate.interp1d(prCurve[1, :], prCurve[0, :])

plotPR = fPR(xVal)

np.savetxt(plotFile, plotPR, fmt='%f')
'''

'''
E = np.load(rawFile)
nESamples = 100;
maxE = 1.0;
step = maxE / nESamples
epsE = step / 100;

E = torch.from_numpy(E)

nPairs = E.size(0)


plot = torch.Tensor(nESamples + 1).fill_(0)

for l in range(nESamples+1):
    
    if l == 0:
        thresh = epsE;
    else:   
        thresh = step*l;
        
    nMatches = torch.nonzero(E <= thresh).size(0)
    
    plot[l] = nMatches / nPairs;

plotError = plot.cpu().numpy();

np.savetxt(plotFile, plotError, fmt='%f')

'''



In [ ]:
'''
ACSpath = '/home/tommy/Dropbox/specialMath/Harmonic/ECHONet/ACSCNN/SHREC19/2/';

ACSsamples = osp.join(osp.join(ACSpath, 'samples'), '{0}.{1}.npy')
ACSmatches = osp.join(osp.join(ACSpath, 'matches'), '{0}.{1}.npy')
ACStestpairs = osp.join(ACSpath, 'test_pairs.npy')
ACStrainpairs = osp.join(ACSpath, 'train_pairs.npy')

n_train = trainS_dataset.__len__()
n_test = testS_dataset.__len__();

test_pairs_S = [];
test_pairs_T = [];
        
train_pairs_S =[];
train_pairs_T =[];

    
for i in range(n_train):

    dataS = trainS_dataset.get(i)
    dataT = trainT_dataset.get(i)
    
    sName = dataS.name;
    tName = dataT.name;
        
    sID = sName[sName[:len(sName)-1].rfind('0')+1:]
    tID = tName[tName[:len(tName)-1].rfind('0')+1:]
    
    sID = int(sID)
    tID = int(tID)

    samples = torch.cat((dataT.sample_idx[..., None], dataS.sample_idx[..., None]), dim=1).cpu().numpy().astype(int)

    p_matches = dataT.pos_pairs.cpu().numpy().astype(int);

    np.save(ACSsamples.format(sID, tID), samples)

    np.save(ACSmatches.format(sID, tID), p_matches)

    train_pairs_S.append(sID);
    train_pairs_T.append(tID);
    
    
for i in range(n_test):

    dataS = testS_dataset.get(i)
    dataT = testT_dataset.get(i)
    
    sName = dataS.name;
    tName = dataT.name;
    
    sID = sName[sName[:len(sName)-1].rfind('0')+1:]
    tID = tName[tName[:len(tName)-1].rfind('0')+1:]
    
    sID = int(sID)
    tID = int(tID)
    samples = torch.cat((dataT.sample_idx[..., None], dataS.sample_idx[..., None]), dim=1).cpu().numpy().astype(int)

    p_matches = dataT.pos_pairs.cpu().numpy().astype(int);

    np.save(ACSsamples.format(sID, tID), samples)

    np.save(ACSmatches.format(sID, tID), p_matches)

    test_pairs_S.append(sID);
    test_pairs_T.append(tID);
    

train_p_S = torch.tensor(train_pairs_S).long()
train_p_T = torch.tensor(train_pairs_T).long()
test_p_S = torch.tensor(test_pairs_S).long()
test_p_T = torch.tensor(test_pairs_T).long()

train_p = torch.cat( (train_p_S[..., None], train_p_T[..., None]), dim=1).cpu().numpy().astype(int)
test_p = torch.cat( (test_p_S[..., None], test_p_T[..., None]), dim=1).cpu().numpy().astype(int)

np.save(ACStestpairs, test_p);
np.save(ACStrainpairs, train_p);
'''